In [2]:
import pandas as pd
import numpy as np
import scipy.optimize as opt

### 10.5

In [25]:
X = np.array([
    [1, 1, 1, 1],
    [1, 1, 0, 1],
    [1, 0, 0, 0],
])

R = {
    (0, 0, 0): 0.6,
    (0, 0, 1): 0.4,
    (0, 1, 0): 0.3,
    (0, 1, 1): 0.7,
    (1, 0, 0): 0.2,
    (1, 0, 1): 0.8,
    (1, 1, 0): 0.1,
    (1, 1, 1): 0.9,
}

def r_density(r, v, g):
    return R[(v, g, r)]

def v_density(v):
    if v == 0:
        return 0
    else:
        return 1

def g_density(g, theta):
    if g == 0:
        return theta[0]
    else:
        return 1 - theta[0]

def s_density(s, g, theta):
    if g == 0:
        return theta[2] if s == 0 else 1 - theta[2]
    else:
        return theta[1] if s == 0 else 1 - theta[1]

def joint_density(theta, x):
    v, g, r, s = x
    return v_density(v) * g_density(g, theta) * r_density(r, v, g) * s_density(s, g, theta)

def neg_log_likelihood(theta, data):
    sum = 0
    for x in data:
        l = joint_density(theta, x)
        sum -= np.log(l)
    return sum

In [29]:
res = opt.minimize(
    neg_log_likelihood,
    x0 = np.array([0.1, 0.2, 0.5]),
    args=(X,),
    bounds=(
        (1e-6, 1-1e-6),
        (1e-6, 1-1e-6),
        (1e-6, 1-1e-6),
    )
)

In [34]:
np.round(res.x, 3)

array([0.333, 0.   , 1.   ])

### 10.6

In [45]:
x1 = np.array([1/2, 0, 0, 1/2])
x2GivenX1 = np.array([
    0.9, 0.1,
    0.3, 0.7,
    0.4, 0.6,
    0.8, 0.2
]).reshape(-1, 2)
x4GivenX2 = np.array([
    0.4, 0.6,
    0.95, 0.05
]).reshape(-1, 2)
x3GivenX2 = np.array([
    1/3, 1/3, 1/3,
    0.8, 0.1, 0.1
]).reshape(-1, 3)

In [39]:
x1 @ x2GivenX1[:,0] * x4GivenX2[0, 1]

0.51

In [43]:
x1 @ x2GivenX1[:,1] * x4GivenX2[1, 1]

0.0075000000000000015

In [46]:
x1 = np.array([1/4, 1/4, 1/4, 1/4])

In [50]:
x1 @ x2GivenX1[:,0] * x3GivenX2[0, 1] * x4GivenX2[0, 1]

0.12

In [51]:
x1 @ x2GivenX1[:,1] * x3GivenX2[1, 1] * x4GivenX2[1, 1]

0.002

In [93]:
from itertools import product

lightness = 1
thickness = 1
joint = []
for (season, fish) in product(range(4), range(2)):
    l = x1[season] * x2GivenX1[season, fish] * x3GivenX2[fish, lightness] * x4GivenX2[fish, thickness]
    joint.append(l)

joint = np.array(joint).reshape(4, 2)

In [100]:
joint.sum(1) / joint.sum(1).sum()

array([0.36987705, 0.13012295, 0.17008197, 0.32991803])